# Exercise: Performance Optimization 2

Optimize the following function.

In [2]:
function work!(A, B, v, N)
    val = 0
    for i in 1:N
        for j in 1:N
            val = mod(v[i],256);
            A[i,j] = B[i,j]*(sin(val)*sin(val)-cos(val)*cos(val));
        end
    end
end

work! (generic function with 1 method)

The (fixed) input is given by:

In [3]:
N = 4000
A = zeros(N,N)
B = rand(N,N)
v = rand(Int, N);

work!(A,B,v,N)

You can benchmark as follows

In [4]:
using BenchmarkTools
@btime work!($A, $B, $v, $N);

  801.315 ms (0 allocations: 0 bytes)


## Optimizations

In [1]:
gethostname()

"ln-0002"

### Analytic optimization

In [5]:
using Test
x = rand()
@test 1-2*cos(x)*cos(x) ≈ sin(x)*sin(x)-cos(x)*cos(x)
@test -cos(2*x) ≈ sin(x)*sin(x)-cos(x)*cos(x)

Test Passed

In [6]:
function work2!(A, B, v, N)
    val = 0
    for i in 1:N
        for j in 1:N
            val = mod(v[i],256);
            A[i,j] = B[i,j]*(-cos(2*val));
        end
    end
end

@btime work2!($A, $B, $v, $N);

  342.049 ms (0 allocations: 0 bytes)


### Analytic + pulling out val computation

In [7]:
function work3!(A, B, v, N)
    val = 0.0
    for i in 1:N
        val = -cos(2*mod(v[i],256))
        for j in 1:N
            A[i,j] = B[i,j]*val;
        end
    end;
end

@btime work3!($A, $B, $v, $N);

  148.668 ms (0 allocations: 0 bytes)


### Analytic + separate out val computation

In [8]:
function work4!(A, B, v, N)
    val = [-cos(2*mod(x,256)) for x in v]
    
    for i in 1:N
        for j in 1:N
            A[i,j] = B[i,j]*val[i];
        end
    end;
end

@btime work4!($A, $B, $v, $N);

  149.229 ms (2 allocations: 31.30 KiB)


### Analytic + separate out val computation + switch order of loops

In [9]:
function work5!(A, B, v, N)
    val = [-cos(2*mod(x,256)) for x in v]
    
    for j in 1:N
        for i in 1:N
            A[i,j] = B[i,j]*val[i];
        end
    end;
end

@btime work5!($A, $B, $v, $N);

  21.587 ms (2 allocations: 31.30 KiB)


### Analytic + separate out val computation + switch order of loops + `@inbounds`

In [10]:
function work6!(A, B, v, N)
    val = [-cos(2*mod(x,256)) for x in v]
    
    @inbounds for j in 1:N
        for i in 1:N
            A[i,j] = B[i,j]*val[i];
        end
    end;
end

@btime work6!($A, $B, $v, $N);

  19.629 ms (2 allocations: 31.30 KiB)


### Analytic + separate out val computation + switch order of loops + `@inbounds` + lookup table

In [11]:
lookup = [ -cos(2*j) for j in 0:255 ]

function work7!(A, B, v, N, lookup)
    @inbounds val = [lookup[mod(x,256)+1] for x in v]
    
    @inbounds for j in 1:N
        for i in 1:N
            A[i,j] = B[i,j]*val[i];
        end
    end;
end

@btime work7!($A, $B, $v, $N, $lookup);

  19.543 ms (2 allocations: 31.30 KiB)


### Analytic + separate out val computation + switch order of loops + `@inbounds` + Multi-threading

In [14]:
using Hwloc
Hwloc.num_physical_cores()

40

In [15]:
Base.Threads.nthreads()

10

In [16]:
using ThreadPinning
pinthreads(:compact)

In [17]:
import Base.Threads: @threads

function work6_threaded!(A, B, v, N)
    val = [-cos(2*mod(x,256)) for x in v]
    
    @inbounds @threads for j in 1:N
        for i in 1:N
            A[i,j] = B[i,j]*val[i];
        end
    end;
end

@btime work6_threaded!($A, $B, $v, $N);

  4.117 ms (63 allocations: 36.73 KiB)


In [18]:
runtime = @belapsed work6_threaded!($A, $B, $v, $N);
perf = N * N * 1e-6 / runtime # MIt/s
println("Performance: $perf MIt/s")

Performance: 3895.011901695742 MIt/s


## Bonus Question: Performance limit?

Look at your final optimized version of `work!`.

* What is conceptually limiting the performance? I.e. is the function compute- or memory-bound?
* How fast can it "theoretically" be? Can you estimate a performance bound?

In [25]:
using STREAMBenchmark
membw = memory_bandwidth(verbose=true, write_allocate=true)

╔══╡ Multi-threaded:
╠══╡ (10 threads)
╟─ COPY:  101507.1 MB/s
╟─ SCALE: 99775.6 MB/s
╟─ ADD:   101038.2 MB/s
╟─ TRIAD: 100720.1 MB/s
╟─────────────────────
║ Median: 100879.1 MB/s
╚═════════════════════


(median = 100879.1, minimum = 99775.6, maximum = 101507.1)

In [30]:
bs = membw.maximum * 1e-3 # [GB/s] max memory bandwidth
flops = 1
traffic = 3 * 8 # [B/iter] in each iteration

I = flops / traffic
perf_membound = round(I * bs * 1000, digits=2)
runtime_membound = N * N * 1e-6 / perf_membound # in s
println("Memory bounded performance: ", perf_membound, " MIt/s")
println("Memory bounded runtime estimate: ", runtime_membound * 1e3, " ms")

Memory bounded performance: 4229.46 MIt/s
Memory bounded runtime estimate: 3.7829888449116433 ms


In [31]:
# comparison
runtime = @belapsed work6_threaded!($A, $B, $v, $N)
@show runtime
@show runtime_membound
ratio = runtime_membound / runtime
println("\nWe achieve about ", round(ratio * 100; digits=1), "% of the (practical) memory bound estimate.")

runtime = 0.004018645
runtime_membound = 0.0037829888449116434

We achieve about 94.1% of the (practical) memory bound estimate.
